## Import libraries

In [1]:
import ee
import geemap

## Create an interactive map

In [2]:
Map = geemap.Map(center=[40, -100], zoom=4)

## Add Earth Engine Python script

In [5]:
# Add Earth Engine dataset
image = ee.Image("USGS/SRTMGL1_003")

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Chapter:      F6.2 Scaling Up in Earth Engine
#  Checkpoint:   F62b
#  Authors:      Jillian M. Deines, Stefania Di Tommaso, Nicholas Clinton, Noel Gorelick
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Load county dataset.
# Filter counties in Indiana, Illinois, and Iowa by state FIPS code.
# Select only the unique ID column for simplicity.
countiesAll = ee.FeatureCollection('TIGER/2018/Counties')
states = ['17', '18', '19']
uniqueID = 'GEOID'
featColl = countiesAll.filter(ee.Filter.inList('STATEFP', states)) \
.select(uniqueID)

print(featColl.size().getInfo())
print(featColl.limit(1).getInfo())
Map.addLayer(featColl)

# Specify years of interest.
startYear = 2001
endYear = 2020

# Climate dataset info.
imageCollectionName = 'IDAHO_EPSCOR/GRIDMET'
bandsWanted = ['pr', 'tmmn', 'tmmx']
scale = 4000

# Export info.
exportFolder = 'GEE_scalingUp'
filenameBase = 'Gridmet_counties_IN_IL_IA_' + str(scale) + 'm_'

# Initiate a loop, in which the variable i takes on values of each year.
for i in range(startYear, endYear, 1):         

    # Load climate collection for that year.
    startDate = str(i) + '-01-01'

    endYear_adj = i + 1
    endDate = str(endYear_adj) + '-01-01'

    imageCollection = ee.ImageCollection(imageCollectionName) \
    .select(bandsWanted) \
    .filterBounds(featColl) \
    .filterDate(startDate, endDate)

    # Get values at feature collection.
    def func_xjw(image):
        def func_opr(f): # remove rows without data 
            time_start = image.get('system:time_start') # add date property
            dte = ee.Date(time_start).format('YYYYMMdd')
            return f.set('date_ymd', dte)

        return image.reduceRegions(
        collection = featColl,
        reducer = ee.Reducer.mean(),         
        tileScale = 1,
        scale = scale
        ).filter(ee.Filter.notNull(bandsWanted)).map(func_opr)

    sampledFeatures = imageCollection.map(func_xjw).flatten()

    # Prepare export: specify properties and filename.
    columnsWanted = [uniqueID] + ['date_ymd'] + bandsWanted
    filename = str(filenameBase) + str(i)

    geemap.ee_export_vector_to_drive(
    collection = sampledFeatures,
    description = filename,
    folder = exportFolder,
    fileFormat = 'CSV',
    selectors = columnsWanted
    )


#  -----------------------------------------------------------------------
#  CHECKPOINT
#  -----------------------------------------------------------------------




293
{'type': 'FeatureCollection', 'columns': {}, 'version': 1566851338350559, 'id': 'TIGER/2018/Counties', 'properties': {'system:asset_size': 57687881}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-89.14440553232701, 38.47386539796727], [-89.14422601609384, 38.473865455658775], [-89.1437322514883, 38.47386539161557], [-89.14368738993991, 38.47386538588098], [-89.1435975939856, 38.47386545404463], [-89.14274484974976, 38.47391033885664], [-89.1402762407341, 38.47391028921294], [-89.13987230534873, 38.473910304003375], [-89.13946836596041, 38.473910317376905], [-89.13919909505006, 38.473910272206446], [-89.13848093363512, 38.4739103083925], [-89.138256523778, 38.47391026878123], [-89.13762815506206, 38.473910315865496], [-89.13664075088421, 38.47395521174677], [-89.13587767425584, 38.47395522319315], [-89.13524934159932, 38.47395517761094], [-89.13511465607063, 38.47395523011729], [-89.13471074226574, 38.47395522733222], [-89.13453118333177, 38.473

## Display the interactive map

In [6]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…